# Import Liblary 

In [2]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from joblib import Parallel, delayed
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from collections import Counter

import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
!pip install imbalanced-learn

[nltk_data] Downloading package punkt to C:\Users\Wildan
[nltk_data]     Septian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Wildan
[nltk_data]     Septian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Memuat Dataset Hasil Scraping

In [3]:
# Memuat dataset
review_df = pd.read_csv("ulasan_efootball2025_full.csv")
review_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,e6b660a4-5978-4918-b13c-dc272ab5af11,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Feel game ini kurang, Tolong perbaiki kontrol ...",1,1932,9.4.0,2025-04-22 01:19:44,NaN,NaN,9.4.0
1,c4e7ea21-f803-4aba-a14e-8f067bd3b499,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Game bola penuh dengan kebusukan, script makin...",1,3,9.4.0,2025-04-30 20:06:44,NaN,NaN,9.4.0
2,0be8bb2f-1a67-403e-b7b3-f261fd0f7509,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sinyal udah 5G (stabil), RAM 12GB dll (khusus ...",1,118,9.4.0,2025-04-27 13:38:51,NaN,NaN,9.4.0
3,5457adb1-370d-4b85-9395-0ab9cdde7ada,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Update yg sekarang kurang memuaskan, banyak Bu...",1,335,9.4.0,2025-04-19 15:19:56,NaN,NaN,9.4.0
4,af325e7a-153e-41e0-b47f-a109a89399ad,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,saya bermain game ini sudah hampir 5 tahun pun...,3,114,9.4.0,2025-04-17 19:56:37,NaN,NaN,9.4.0


# Data Preprocessing

In [4]:
#Melihat tipe data,banyaknya kolom dan baris dari dataset
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   reviewId              15000 non-null  object 
 1   userName              15000 non-null  object 
 2   userImage             15000 non-null  object 
 3   content               15000 non-null  object 
 4   score                 15000 non-null  int64  
 5   thumbsUpCount         15000 non-null  int64  
 6   reviewCreatedVersion  13042 non-null  object 
 7   at                    15000 non-null  object 
 8   replyContent          0 non-null      float64
 9   repliedAt             0 non-null      float64
 10  appVersion            13042 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1.3+ MB


In [5]:
# Cek data duplikat
print("Jumlah Duplikasi data : " ,review_df.duplicated().sum())

Jumlah Duplikasi data :  0


In [6]:
# menghapus kolom yang kosong dan tidak relevan
review_df = review_df.drop(columns=["replyContent","repliedAt"])

In [7]:
review_df.info()
review_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              15000 non-null  object
 1   userName              15000 non-null  object
 2   userImage             15000 non-null  object
 3   content               15000 non-null  object
 4   score                 15000 non-null  int64 
 5   thumbsUpCount         15000 non-null  int64 
 6   reviewCreatedVersion  13042 non-null  object
 7   at                    15000 non-null  object
 8   appVersion            13042 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.0+ MB


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
0,e6b660a4-5978-4918-b13c-dc272ab5af11,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Feel game ini kurang, Tolong perbaiki kontrol ...",1,1932,9.4.0,2025-04-22 01:19:44,9.4.0
1,c4e7ea21-f803-4aba-a14e-8f067bd3b499,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Game bola penuh dengan kebusukan, script makin...",1,3,9.4.0,2025-04-30 20:06:44,9.4.0
2,0be8bb2f-1a67-403e-b7b3-f261fd0f7509,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sinyal udah 5G (stabil), RAM 12GB dll (khusus ...",1,118,9.4.0,2025-04-27 13:38:51,9.4.0
3,5457adb1-370d-4b85-9395-0ab9cdde7ada,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Update yg sekarang kurang memuaskan, banyak Bu...",1,335,9.4.0,2025-04-19 15:19:56,9.4.0
4,af325e7a-153e-41e0-b47f-a109a89399ad,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,saya bermain game ini sudah hampir 5 tahun pun...,3,114,9.4.0,2025-04-17 19:56:37,9.4.0


In [8]:
# Menampilkan distribusi jumlah masing-masing nilai pada kolom 'score'
print("\nSebaran skor:")
print(review_df['score'].value_counts())


Sebaran skor:
score
1    6033
5    3712
3    1806
4    1804
2    1645
Name: count, dtype: int64


In [9]:
stop_words = set(stopwords.words('indonesian'))

def bersihkan_teks(teks):
    teks = teks.lower()  # huruf kecil semua
    teks = re.sub(r"http\S+|www\S+|https\S+", '', teks)  # hapus URL
    teks = re.sub(r'@\w+|#\w+', '', teks)  # hapus mention dan hashtag
    teks = re.sub(r'\d+', '', teks)  # hapus angka
    teks = teks.translate(str.maketrans('', '', string.punctuation))  # hapus tanda baca
    teks = teks.strip()  # hapus spasi di awal/akhir
    teks = re.sub(r'\s+', ' ', teks)  # ganti multiple spasi dengan satu
    teks = ' '.join([word for word in teks.split() if word not in stop_words])  # hapus stopword
    return teks
review_df['cleaned_content'] = review_df['content'].apply(bersihkan_teks)

In [10]:
# Pelabelan Sentimen
from textblob import TextBlob

# Fungsi untuk menentukan label sentimen dari sebuah teks
def tentukan_sentimen(teks):
    polaritas = TextBlob(teks).sentiment.polarity  # Hitung skor polaritas dari teks
    if polaritas > 0:
        return 'positif'  # Jika polaritas lebih dari 0, labelkan sebagai positif
    elif polaritas < 0:
        return 'negatif'  # Jika polaritas kurang dari 0, labelkan sebagai negatif
    else:
        return 'netral'   # Jika polaritas sama dengan 0, labelkan sebagai netral

# Terapkan fungsi sentimen ke kolom 'cleaned_content' dan simpan hasilnya di kolom 'label'
review_df['label'] = review_df['cleaned_content'].apply(tentukan_sentimen)

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encode label (sentimen) menjadi angka
y = LabelEncoder().fit_transform(review_df['label'])
X = review_df['cleaned_content']

# Pisahkan data menjadi data pelatihan dan pengujian
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Pembuatan Model

## SVM

In [12]:
# Preprocessing untuk model berbasis TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train_raw)
X_test_tfidf = vectorizer.transform(X_test_raw)

# Terapkan SMOTE untuk mengatasi masalah ketidakseimbangan kelas
smote = SMOTE(random_state=42)
X_train_tfidf_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

print("Distribusi label setelah SMOTE:", Counter(y_train_smote))

# Model SVM
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train_tfidf_smote, y_train_smote)
y_pred_svm = svm.predict(X_test_tfidf)

# Evaluasi model SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("\nEvaluasi Model SVM:")
print(f"Accuracy: {accuracy_svm}")
print(classification_report(y_test, y_pred_svm, target_names=['negatif', 'netral', 'positif']))

Distribusi label setelah SMOTE: Counter({0: 4898, 1: 4898, 2: 4898})

Evaluasi Model SVM:
Accuracy: 0.9573333333333334
              precision    recall  f1-score   support

     negatif       0.97      0.95      0.96      1225
      netral       0.94      0.99      0.97      1125
     positif       0.95      0.90      0.93       650

    accuracy                           0.96      3000
   macro avg       0.96      0.95      0.95      3000
weighted avg       0.96      0.96      0.96      3000



## LogisticRegression

In [13]:
from gensim.models import Word2Vec
# Tokenisasi untuk Word2Vec
sentences = [text.split() for text in review_df['cleaned_content']]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Mengonversi teks menjadi vektor menggunakan Word2Vec
def get_word2vec_vectors(texts, model):
    vectors = []
    for text in texts:
        words = text.split()
        vec = np.zeros(100)
        count = 0
        for word in words:
            try:
                vec += model[word]
                count += 1
            except KeyError:
                continue
        if count > 0:
            vec /= count
        vectors.append(vec)
    return np.array(vectors)

X_train_w2v = get_word2vec_vectors(X_train_raw, w2v_model.wv)
X_test_w2v = get_word2vec_vectors(X_test_raw, w2v_model.wv)

# Terapkan SMOTE untuk mengatasi masalah ketidakseimbangan kelas
X_train_w2v_smote, y_train_smote = smote.fit_resample(X_train_w2v, y_train)

In [15]:
from sklearn.linear_model import LogisticRegression

# Model Logistic Regression
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
lr.fit(X_train_w2v_smote, y_train_smote)
y_pred_lr = lr.predict(X_test_w2v)
# Evaluasi model Logistic Regression
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("\nEvaluasi Model Logistic Regression:")
print(f"Accuracy: {accuracy_lr}")
print(classification_report(y_test, y_pred_lr, target_names=['negatif', 'netral', 'positif']))

C:\Users\Wildan Septian\anaconda3\envs\sentimen_analisis\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



Evaluasi Model Logistic Regression:
Accuracy: 0.7943333333333333
              precision    recall  f1-score   support

     negatif       0.92      0.82      0.87      1225
      netral       0.77      0.83      0.80      1125
     positif       0.64      0.68      0.66       650

    accuracy                           0.79      3000
   macro avg       0.78      0.78      0.78      3000
weighted avg       0.80      0.79      0.80      3000



In [16]:
from sklearn.model_selection import GridSearchCV


# Terapkan SMOTE untuk mengatasi masalah ketidakseimbangan kelas
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'saga'],
    'max_iter': [500, 1000, 2000]
}

grid = GridSearchCV(LogisticRegression(multi_class='multinomial', random_state=42),
                    param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid.fit(X_train_w2v_smote, y_train_smote)

print("Best parameters:", grid.best_params_)
best_lr = grid.best_estimator_
y_pred_tuned = best_lr.predict(X_test_w2v)

# Evaluasi ulang
print("\nEvaluasi Model Logistic Regression (Tuned):")
print(classification_report(y_test, y_pred_tuned, target_names=['negatif', 'netral', 'positif']))

C:\Users\Wildan Septian\anaconda3\envs\sentimen_analisis\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Best parameters: {'C': 100, 'max_iter': 2000, 'solver': 'lbfgs'}

Evaluasi Model Logistic Regression (Tuned):
              precision    recall  f1-score   support

     negatif       0.95      0.88      0.91      1225
      netral       0.83      0.89      0.86      1125
     positif       0.71      0.72      0.72       650

    accuracy                           0.85      3000
   macro avg       0.83      0.83      0.83      3000
weighted avg       0.85      0.85      0.85      3000



## LSTM

In [17]:
# Tokenisasi untuk LSTM
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train_raw)

X_train_seq = tokenizer.texts_to_sequences(X_train_raw)
X_test_seq = tokenizer.texts_to_sequences(X_test_raw)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# One-hot encoding untuk label
y_train_cat = pd.get_dummies(y_train).values
y_test_cat = pd.get_dummies(y_test).values

# Model LSTM
# Model LSTM
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model_lstm.add(LSTM(128))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(3, activation='softmax'))



model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Callback untuk menghentikan pelatihan lebih awal jika tidak ada perbaikan
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)
]

history = model_lstm.fit(
    X_train_pad, y_train_cat,
    epochs=15,
    batch_size=32,
    validation_data=(X_test_pad, y_test_cat),
    callbacks=callbacks
)

# Evaluasi model LSTM
loss, accuracy_lstm = model_lstm.evaluate(X_test_pad, y_test_cat)
print("\nEvaluasi Model LSTM:")
print(f"Accuracy: {accuracy_lstm}")

C:\Users\Wildan Septian\anaconda3\envs\sentimen_analisis\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 27s 66ms/step - accuracy: 0.6758 - loss: 0.7689 - val_accuracy: 0.9493 - val_loss: 0.1819 - learning_rate: 0.0010
Epoch 2/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 25s 68ms/step - accuracy: 0.9620 - loss: 0.1397 - val_accuracy: 0.9587 - val_loss: 0.1724 - learning_rate: 0.0010
Epoch 3/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 28s 74ms/step - accuracy: 0.9827 - loss: 0.0705 - val_accuracy: 0.8900 - val_loss: 0.3791 - learning_rate: 0.0010
Epoch 4/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 26s 69ms/step - accuracy: 0.9807 - loss: 0.0781 - val_accuracy: 0.9667 - val_loss: 0.1401 - learning_rate: 0.0010
Epoch 5/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 27s 71ms/step - accuracy: 0.9913 - loss: 0.0397 - val_accuracy: 0.9680 - val_loss: 0.1315 - learning_rate: 0.0010
Epoch 6/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 25s 67ms/step - accuracy: 0.9925 - loss: 0.0255 - val_accuracy: 0.9717 - val_loss: 0.1251 - learning_rate: 0.0010
Epoch 7/15
375/375 ━━━━━━━━━━━━━━━━━━━━ 26s 70ms/step - accuracy: 0.9954 - l